# 2. Сборка датасета 

In [1]:
import os
import sys

import pandas as pd
import numpy as np
import random

from matplotlib import pyplot as plt

from tqdm import tqdm

from pandarallel import pandarallel

import pymorphy2
import nltk
import pickle

from sklearn.model_selection import train_test_split

SEED = 1
random.seed(SEED)

pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## 1. Загрузка данных 

In [2]:
abbr = pd.read_csv("../data/abbr.csv")
lenta = pd.read_csv("../data/lenta.csv")

In [3]:
abbr

,abbr,desc,desc_norm,desc_len,abbr_norm,abbr_len,abbr_count,desc_count,abbr_id
0,замгенпрокурора,заместитель генерального прокурора,заместитель генеральный прокурор,3,замгенпрокурора,1,29,43,0
1,...быт...,бытовой,бытовой,1,быт,1,121734,397,1
2,РС,реактивный снаряд,реактивный снаряд,2,рс,1,67397,28,2
3,МЦ,морская цель,морской цель,2,мц,1,394,10,3
4,СКОВС,Северо-Кавказский окружной военный суд,северо кавказский окружный военный суд,5,сковс,1,8383,17,4
...,...,...,...,...,...,...,...,...,...
5620,ГО,городское отделение,городской отделение,2,го,1,293935,17,5620
5621,НАО,Ненецкий автономный округ,ненецкий автономный округ,3,нао,1,340,122,5621
5622,инстр.,инструкция,инструкция,1,инстр,1,1088,221,5622
5623,гост.,гостиница,гостиница,1,гост,1,3010,997,5623


## 2. Построение дерева сокращений

In [4]:
class AbbrInfo:
    def __init__(self, abbr_id, abbr, abbr_count):
        self.abbr_id = abbr_id 
        self.abbr = abbr
        self.abbr_count = abbr_count

ABBR_LIST_KEY = "<ABBR_LIST_KEY>"
        
def create_abbr_tree(abbr, abbr_list_key = ABBR_LIST_KEY):        
    tree = {}
    for norm_desc, norm_abbr, abbr_id, abbr_count in abbr[["desc_norm", 
                                                           "abbr_norm", 
                                                           "abbr_id", 
                                                           "abbr_count"]].values:
        words = norm_desc.split(" ")

        curr_tree = tree
        for word in words:
            if word not in curr_tree:
                curr_tree[word] = {}
            curr_tree = curr_tree[word]


        if abbr_list_key not in curr_tree:
            curr_tree[abbr_list_key] = []

        curr_tree[abbr_list_key].append(AbbrInfo(abbr_id, norm_abbr, abbr_count))
    return tree

In [5]:
abbr_tree = create_abbr_tree(abbr)
abbr_tree["министерство"]

{'образование': {'и': {'наука': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb41bc8e0>]}}},
 'труд': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb41d2280>]},
 'оборона': {'украина': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb41e6460>]},
  '<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb40a09a0>,
   <__main__.AbbrInfo at 0x7fceb3f38700>]},
 'экономический': {'развитие': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb417f280>]}},
 'связь': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb41b0940>]},
 'финансы': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb414f0a0>]},
 'внутренний': {'безопасность': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb414f400>]},
  'дело': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb3ff3340>]}},
 'иностранный': {'дело': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb4162a00>,
    <__main__.AbbrInfo at 0x7fceb4162a60>]}},
 'строительство': {'<ABBR_LIST_KEY>': [<__main__.AbbrInfo at 0x7fceb4170ee0>]},
 'юстиция': {'<ABBR_LIST

## 3. Получение меток для токенов

In [6]:
def choice_abbr(abbr_list: list, 
                weighted_choice: bool = True, 
                add_to_zeros: float = 0):
    abbr_counts = []
    
    if weighted_choice:
        for abbr_info in abbr_list:
            cnt = abbr_info.abbr_count
            if cnt == 0:
                cnt = add_to_zeros
            abbr_counts.append(cnt)
    else:
        abbr_counts = None
    
    
    return random.choices(abbr_list, weights=abbr_counts, k=1)[0]

In [7]:
OUTSIDE_LABEL = "_"
BEGIN_LABEL = "B"
END_LABEL = "E"
INSIDE_LABEL = "I"
ONE_WORD_LABEL = "W"

def get_text_labels(text, 
                    abbr_tree, 
                    weighted_choice: bool = None, 
                    add_to_zeros: float = None):
    text = text.split(" ")
    labels = [OUTSIDE_LABEL for i in range(len(text))]

    curr_node = abbr_tree
    desc_start = None

    word_i = 0
    while word_i < len(text):
        curr_i = word_i
        while curr_i < len(text) and text[curr_i] in curr_node:
            curr_node = curr_node[text[curr_i]]
            curr_i += 1

        if ABBR_LIST_KEY in curr_node: 

            abbr_id = choice_abbr(curr_node[ABBR_LIST_KEY], weighted_choice, add_to_zeros).abbr_id

            labels[word_i] = f"{BEGIN_LABEL}-{abbr_id}"
            for j in range(word_i + 1, curr_i - 1): 
                labels[j] = f"{INSIDE_LABEL}-{abbr_id}"
            labels[curr_i - 1] = f"{END_LABEL}-{abbr_id}"

            if word_i == curr_i - 1:
                labels[word_i] = f"{ONE_WORD_LABEL}-{abbr_id}"

            word_i = curr_i - 1

        curr_node = abbr_tree
        word_i += 1
    return " ".join(labels)

get_text_labels(lenta.text_norm.iloc[0], abbr_tree)

'_ _ _ _ _ _ _ _ _ _ _ W-2957 _ _ W-4400 W-842 _ _ W-1523 _ W-842 _ _ _ W-3458 W-5182 _ _ _ _ W-2547 _ _ _ _ _ W-3833 _ _ _ _ W-1523 _ _ _ _ _ _ W-4156 W-1420 _ _ W-1471 _ _ _ _ _ _ W-5412 _ W-4589 _ _ _ _ _ _ W-3305 _ W-2821 _ _ _ _ _ _ _ _ W-2698 _ _ W-4782 _ W-1978 _ _ W-231 _ W-2216 W-2018 W-4782 _ W-3435 _ _ _ _ W-4210 _ _ W-1940 _ _ _ _ _ _ _ _ _ _ W-1002 _ _ W-1454 _ W-2568'

In [8]:
from ipymarkup         import show_box_markup
from ipymarkup.palette import palette, PALETTE, BLUE, RED, GREEN, PURPLE, BROWN, ORANGE

def show_markup(recipe,  tags, use_abbr: bool = True):
    mapper = lambda tag: tag[2:] if "-" in tag else tag
    
    tags  = [mapper(tag) for tag in tags]
    text  = ' '.join(recipe)
    spans = []
        
    start, end, tag = 0, len(recipe[0]), tags[0]
    
    for word, ttag in zip(recipe[1:], tags[1:]): 
        
        if tag == ttag:
            end  += 1 + len(word)
            
        else:
            span  = (start, end, tag)
            spans.append(span)
        
            start = 1 + end
            end  += 1 + len(word)
            
            if ttag != "_":
                if use_abbr:
                    label = abbr[abbr.abbr_id == int(ttag)].abbr_norm.iloc[0]
                else:
                    label = abbr[abbr.abbr_id == int(ttag)].desc_norm.iloc[0]
                ttag = label + f" ({ttag})"
            tag   = ttag
            
    span  = (start, end, tag)
    spans.append(span)        
            
    show_box_markup(text, spans)

In [9]:
text = lenta.text_norm.iloc[6]
labels = get_text_labels(text, abbr_tree)

show_markup(text.split(" "), labels.split(" "))

In [10]:
lenta["labels"] = lenta["text_norm"].parallel_apply(lambda x: get_text_labels(x, abbr_tree))

## 4. Замена слов на сокращения 

In [11]:
def replace_word_by_abbr(text, labels, abbr, p_replace: float = 0.5):
    text = text.split(" ")
    labels = labels.split(" ")
    
    new_text = []
    new_labels = []

    i = 0
    while i < len(text):
        label = labels[i]
        if label == OUTSIDE_LABEL:
            new_text.append(text[i])
            new_labels.append(OUTSIDE_LABEL)

        mode = label[0]

        if mode in [ONE_WORD_LABEL, BEGIN_LABEL]:
            abbr_id = int(label[2:])
            replaced = random.choices([False, True], weights=[(1 - p_replace), p_replace])[0]
            if replaced:
                norm_abbr = abbr[abbr.abbr_id == abbr_id].abbr_norm.iloc[0].split(" ")
                
                if len(norm_abbr) == 1:
                    new_text.append(norm_abbr[0])
                    new_labels.append(f"{ONE_WORD_LABEL}-{str(abbr_id)}")
                else:
                    new_text.append(norm_abbr[0])
                    new_labels.append(f"{BEGIN_LABEL}-{str(abbr_id)}")
                    for word in norm_abbr[1:-1]:
                        new_text.append(word)
                        new_labels.append(f"{INSIDE_LABEL}-{str(abbr_id)}")
                    new_text.append(norm_abbr[-1])
                    new_labels.append(f"{END_LABEL}-{str(abbr_id)}")

            while i < len(text) and labels[i] != OUTSIDE_LABEL and int(labels[i][2:]) == abbr_id:
                if not replaced:
                    new_text.append(text[i])
                    new_labels.append(OUTSIDE_LABEL)
                i += 1
        else:
            i += 1

    new_text = " ".join(new_text)
    new_labels = " ".join(new_labels)
    
    return pd.Series({"new_text": new_text, "new_labels": new_labels})

In [12]:
for i in range(10, 15):
    text = lenta.text_norm.iloc[i]
    labels = get_text_labels(text, abbr_tree)

    replaced_series = replace_word_by_abbr(text, labels, abbr)


    show_markup(replaced_series["new_text"].split(" "), 
                replaced_series["new_labels"].split(" "), 
                use_abbr=False)

In [13]:
lenta[["text_new", "labels_new"]] = (
    lenta[["text_norm", "labels"]]
        .parallel_apply(lambda x: replace_word_by_abbr(x["text_norm"], x["labels"], 
                                                       abbr, p_replace=0.5), axis=1)
)

In [15]:
lenta

,text,text_norm,text_len,labels,text_new,labels_new
0,"Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значит...",бой у сопоцкина и друскеник закончиться отступление германец неприятель приблизиться с север к осовцу начать артиллерийский борьба с крепость в артиллерийский бой принимать участие тяжёлый калибр с ранний утро 14 сентябрь огонь достигнуть значительный...,118,_ _ _ _ _ _ _ _ _ _ _ W-2957 _ _ W-4400 W-842 _ _ W-1523 _ W-842 _ _ _ W-3458 W-5181 _ _ _ _ W-1454 _ _ _ _ _ W-3833 _ _ _ _ W-1523 _ _ _ _ _ _ W-4156 W-1420 _ _ W-1471 _ _ _ _ _ _ W-5412 _ W-4590 _ _ _ _ _ _ W-3305 _ W-2821 _ _ _ _ _ _ _ _ W-2698 _ _...,бой у сопоцкина и друскеник закончиться отступление германец неприятель приблизиться с север к осовцу начать арт борьба с крепость в артиллерийский бой принимать участие тяжёлый кал с ранний утро 14 сентябрь огонь достигнуть значительный напряжение по...,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-842 _ _ _ _ _ _ _ _ _ W-5181 _ _ _ _ _ _ _ _ _ _ W-3833 _ _ _ _ W-1523 _ _ _ _ _ _ W-4156 _ _ _ W-1471 _ _ _ _ _ _ _ _ W-4590 _ _ _ _ _ _ _ _ W-2821 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-1891 _ W-2216 _ W-4782 _ W-3435 _ _ ...
1,"Фотограф-корреспондент Daily Mirror рассказывает случай, который порадует всех друзей животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки п...",фотограф корреспондент daily mirror рассказывать случай который порадовать весь друг животное лейтенант бельгийский артиллерия руководить оборона один из фортов льеж ни за что не хотеть расстаться с свой собака бульдог когда пруссак пробраться между ф...,99,_ W-805 _ _ _ _ W-1503 _ _ _ _ W-5020 _ W-1940 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-4935 _ _ _ W-2010 _ _ _ _ W-1503 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-3609 _ _ _ _ _ _ W-5029 W-3833 _ _ _ _ _ _ _ _ _ W-4074 _ _ W-254...,фотограф корреспондент daily mirror рассказывать случай который порадовать весь друг животное лейтенант бельгийский артиллерия руководить оборона один из фортов льеж ни за что не хотеть расстаться с свой собака бульдог когда пруссак пробраться между ф...,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-2010 _ _ _ _ W-1503 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-3609 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-4074 _ _ _ _ W-2568
2,"Лица, приехавшие в Варшаву из Люблина, передают, что туда доставлен «швабский зверь» — взятый в плен прусский майор Прейскер, бывший комендант Калиша. Это — здоровый детина, с типично прусским наглым лицом и мутными глазами. Когда этого «зверя» вели с...",лицо приехать в варшава из люблин передавать что туда доставить швабский зверь взять в плен прусский майор прейскер бывший комендант калиша это здоровый детина с типично прусский наглый лицо и мутный глаз когда это зверь вели с партия пленный по улица...,112,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-1975 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-4667 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-844 _ _ _ _ _ _ _ W-1002 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-2297 _ _ W-3851 _ _ _ W-3943 W-3888 _ _ _ _ W-40...,лицо приехать в варшава из люблин передавать что туда доставить швабский зверь взять в плен прусский майор прейскер бывш комендант калиша это здоровый детина с типично прусский наглый лицо и мутный глаз когда это зверь вели с партия пленный по ул и из...,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-1975 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-4667 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ W-2297 _ _ _ _ _ _ _ W-3888 _ _ _ _ _ _ _ _ _ _ _ W-2568
3,"Как стало известно агентству Ассошиэйтед Пресс, экипаж последней экспедиции на станцию ""Мир"" считает ее способной выйти из-под контро

## 5. Резделение на обучение и тест 

In [16]:
lenta_train, lenta_test = train_test_split(lenta, test_size=0.2, shuffle=True, random_state=SEED)

## 6. Сохранение данных

In [17]:
lenta_train[["text_norm", "text_new", "labels_new"]].to_csv("../data/lenta_train.csv", index=False, header=True)
lenta_test[["text_norm", "text_new", "labels_new"]].to_csv("../data/lenta_test.csv", index=False, header=True)